In [83]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pysal.explore import inequality
from inequality.gini import Gini_Spatial
from pysal.lib import weights
import warnings
from shapely.geometry import Polygon
import osmnx as ox
city_geo = ox.geocoder.geocode_to_gdf

In [69]:
ls_in = ['Abidjan','Addis Ababa','Baghdad','Bangalore','Belgrade','Belo Horizonte','Bogota','Bologna','Cali','Canberra',\
         'Casablanca','Chandigarh','Chelyabinsk','Chengdu','Christchurch','Cochabamba','Copenhagen','Curitiba','Damascus',\
         'Detroit','Dhaka','Dongguan','Fortaleza','Ghent','Guatemala City','Gwangju','Hanoi','Indore','Johannesburg',\
         'Kampala','Karlsruhe','Kathmandu','Kharkiv','Kuala Lumpur','Kumamoto','Lagos','Leicester','Liverpool',\
         'Lubumbashi','Lviv','Manila','Memphis','Mexico City','Mombasa','Nanjing','New Delhi','Oran','Oslo','Perth',\
         'Philadelphia','Pune','Rennes','Rosario','Rostov','San Antonio','San Diego','Santo Domingo','Seattle','Seoul',\
         'Sevilla','Shijiazhuang','Shiraz','Skopje','Surabaya','Tallinn','Tashkent','Tel Aviv','Toronto','Turin',\
         'Utrecht','Utsunomiya','Washington DC','Yaounde','Yerevan','Zibo']

In [70]:
len(ls_in)

ls_adj = ['Canberra','Karlsruhe','Oslo']

In [71]:
cities = pd.read_excel('cities.xlsx')
cities_adj = cities[cities['City'].isin(ls_adj)]
cities_adj = cities_adj.sort_values('City').reset_index()


cities_adj2 = cities[cities['City'].isin(ls_adj)]
cities_adj2 = cities_adj2.sort_values('City').reset_index()

In [86]:
%%time
path = "D:/Dumps/GEE-WP Scores/Gravity/100m grids/Grid_geoms_adj/"
ls_file = os.listdir(path)
ls_file = np.array(ls_file)[[i.rsplit('_')[1] == 'grav3' for i in ls_file]]
ls_file = ls_file[[any([j in i for j in ls_adj]) for i in ls_file]]
ls_file = np.array(ls_file)[[any([j in i for j in ls_adj]) for i in ls_file]]

#ls_sub = np.array(ls_file)[ls_bool]
#ls_out = np.array(ls_file)[~np.array(ls_bool)]

#area = [city_geo(i.split(', ')).dissolve().geometry for i in cities_adj2['OSM_ineq_adj']]

#j = 0
#k = 0
city_score = []
for i in range(len(ls_file)):
    #if ls_bool[i]:
    file = gpd.read_file(path+ls_file[i])
    
    minx = file.bounds.minx
    maxx = file.bounds.maxx
    miny = file.bounds.miny
    maxy = file.bounds.maxy

    # Some geometries result in a multipolygon when overlaying (like i.e. 0.05 meters), coords error.
    # Therefore recreate the polygon.
    Poly = []
    for k in range(len(file)):
        Poly.append(Polygon([(minx[k],maxy[k]),(maxx[k],maxy[k]),(maxx[k],miny[k]),(minx[k],miny[k])]))
    file.geometry = Poly
    
    city_score.append(file)
    
    print(cities_adj['City'][i])
        
   #     print(ls_sub[j].split('_')[-1].split('.')[0])
    #    j = j+1
    #else:
    #    city_score.append(gpd.read_file(path+ls_out[k]))
    #    
    #    print(ls_out[k].split('_')[-1].split('.')[0])
    #    k = k+1

Canberra
Karlsruhe
Oslo
CPU times: total: 1min 34s
Wall time: 1min 34s


In [87]:
city_score[2].geom_type.value_counts()

Polygon    109716
dtype: int64

In [88]:
%%time
warnings.filterwarnings('ignore')
Gcities_gini = []
Gcities_spatialgini = []
for city in enumerate(city_score):
    Gtr_gini = []
    Gtr_spatialgini = []
    print(ls_file[city[0]].split('.')[0].split('_')[-1])
    for i in city[1][['tr_300','tr_600','tr_1000']]:
        float_df = city[1][city[1][i].isna() == False][['geometry','population',i]]

        gini = pd.DataFrame(float_df[i].value_counts()).sort_index()
        gini['pop'] = float_df.groupby(i)['population'].sum()
        gini = gini.reset_index()
        gini['frac_pop'] = gini['pop'] / gini['pop'].sum()
        gini['richer_frac'] = 1-gini['frac_pop'].cumsum()
        gini['ugs'] = (gini['index'] * gini['pop']) 
        gini['frac_ugs'] = gini.ugs/gini.ugs.sum()
        gini_score = 1-(gini.frac_ugs*(gini.frac_pop+2*gini.richer_frac)).sum()
        print('gini',i.rsplit('_')[-1],round(gini_score,4))
        Gtr_gini.append(gini_score)
        
        wq = weights.Queen.from_dataframe(float_df)
        wq.transform = "B"
        spatial_gini = Gini_Spatial(float_df[i], wq).g
        print('spatial gini',i.rsplit('_')[-1],round(spatial_gini,4))
        Gtr_spatialgini.append(spatial_gini)
    Gcities_gini.append(Gtr_gini)
    Gcities_spatialgini.append(Gtr_spatialgini)
    print(' ')

Canberra
gini 300 0.7573
spatial gini 300 0.7151
gini 600 0.6745
spatial gini 600 0.6475
gini 1000 0.5713
spatial gini 1000 0.5598
 
Karlsruhe
gini 300 0.7592
spatial gini 300 0.6877
gini 600 0.6219
spatial gini 600 0.5784
gini 1000 0.4976
spatial gini 1000 0.482
 
Oslo
gini 300 0.8751
spatial gini 300 0.8769
gini 600 0.7218
spatial gini 600 0.7298
gini 1000 0.5839
spatial gini 1000 0.5954
 
CPU times: total: 3min 34s
Wall time: 3min 35s


In [89]:
gini_df = pd.DataFrame(Gcities_gini, columns = ['Gini G-300','Gini G-600','Gini G-1000'])
spatialgini_df = pd.DataFrame(Gcities_spatialgini, columns = ['S-gini G-300','S-gini G-600','S-gini G-1000'])
comb_gini = pd.concat([gini_df, spatialgini_df], axis = 1)
comb_gini.index = [i.split('_')[-1].split('.')[0] for i in ls_file]
comb_gini

,Gini G-300,Gini G-600,Gini G-1000,S-gini G-300,S-gini G-600,S-gini G-1000
Canberra,0.757310,0.674484,0.571259,0.715117,0.647454,0.559838
Karlsruhe,0.759177,0.621880,0.497628,0.687741,0.578448,0.482046
Oslo,0.875084,0.721834,0.583930,0.876949,0.729761,0.595352


In [90]:
comb_gini2 = comb_gini
comb_gini.sort_values(by='S-gini G-300')

,Gini G-300,Gini G-600,Gini G-1000,S-gini G-300,S-gini G-600,S-gini G-1000
Karlsruhe,0.759177,0.621880,0.497628,0.687741,0.578448,0.482046
Canberra,0.757310,0.674484,0.571259,0.715117,0.647454,0.559838
Oslo,0.875084,0.721834,0.583930,0.876949,0.729761,0.595352


In [91]:
comb_gini.to_csv('D:/Dumps/GEE-WP Scores/Gravity/gini_scores_2.csv')

In [112]:
%%time
path = "D:/Dumps/GEE-WP Scores/E2SFCA/100m grids/Grid_geoms_adj/"
ls_file = os.listdir(path)
ls_file = np.array(ls_file)[[i.rsplit('.')[-1] == 'gpkg' for i in ls_file]]
ls_file = ls_file[[any([j in i for j in ls_adj]) for i in ls_file]]
ls_file = np.array(ls_file)[[any([j in i for j in ls_adj]) for i in ls_file]]

#ls_sub = np.array(ls_file)[ls_bool]
#ls_out = np.array(ls_file)[~np.array(ls_bool)]

#area = [city_geo(i.split(', ')).dissolve().geometry for i in cities_adj2['OSM_ineq_adj']]

#j = 0
#k = 0
city_score = []
for i in range(len(ls_file)):
    #if ls_bool[i]:
    #if ls_bool[i]:
    file = gpd.read_file(path+ls_file[i])
    
    minx = file.bounds.minx
    maxx = file.bounds.maxx
    miny = file.bounds.miny
    maxy = file.bounds.maxy

        # Some geometries result in a multipolygon when dissolving (like i.e. 0.05 meters), coords error.
        # Therefore recreate the polygon.
    Poly = []
    for k in range(len(file)):
        Poly.append(Polygon([(minx[k],maxy[k]),(maxx[k],maxy[k]),(maxx[k],miny[k]),(minx[k],miny[k])]))
    file.geometry = Poly
    
    city_score.append(file)
    
    print(cities_adj['City'][i])
   #     print(ls_sub[j].split('_')[-1].split('.')[0])
    #    j = j+1
    #else:
    #    city_score.append(gpd.read_file(path+ls_out[k]))
    #    
    #    print(ls_out[k].split('_')[-1].split('.')[0])
    #    k = k+1

Canberra
Karlsruhe
Oslo
CPU times: total: 25.6 s
Wall time: 26.7 s


In [113]:
%%time
# For loop city/GINI
# First non-spatial GINI for the 50 cities.

# Queen contignuity
# Spatial GINI - value cell itself
# Weight is values of surrounding cells
# Near and far, with weights of 1 for near and 0 for far.

warnings.filterwarnings('ignore')
cities_gini = []
cities_spatialgini = []
for city in enumerate(city_score):
    tr_gini = []
    tr_spatialgini = []
    print(ls_file[city[0]].split('.')[0])
    for i in city[1].columns[city[1].columns.str.contains('Sc-access')]:
        float_df = city[1][['geometry','population',i]]

        gini = pd.DataFrame(float_df[i].value_counts()).sort_index()
        gini['pop'] = float_df.groupby(i)['population'].sum()
        gini = gini.reset_index()
        gini['frac_pop'] = gini['pop'] / gini['pop'].sum()
        gini['richer_frac'] = 1-gini['frac_pop'].cumsum()
        gini['ugs'] = (gini['index'] * gini['pop']) 
        gini['frac_ugs'] = gini.ugs/gini.ugs.sum()
        gini_score = 1-(gini.frac_ugs*(gini.frac_pop+2*gini.richer_frac)).sum()
        print('gini',i.rsplit(' ')[-1],round(gini_score,4))
        tr_gini.append(gini_score)
        
        wq = weights.Queen.from_dataframe(float_df)
        wq.transform = "B"
        spatial_gini = Gini_Spatial(float_df[i], wq).g
        print('spatial gini',i.rsplit(' ')[-1],round(spatial_gini,4))
        tr_spatialgini.append(spatial_gini)
    cities_gini.append(tr_gini)
    cities_spatialgini.append(tr_spatialgini)
    print(' ')

Canberra
gini 300 0.8477
spatial gini 300 0.8216
gini 600 0.7592
spatial gini 600 0.7394
gini 1000 0.688
spatial gini 1000 0.6739
 
Karlsruhe
gini 300 0.6954
spatial gini 300 0.6679
gini 600 0.5952
spatial gini 600 0.6071
gini 1000 0.5057
spatial gini 1000 0.5483
 
Oslo
gini 300 0.8702
spatial gini 300 0.8864
gini 600 0.7699
spatial gini 600 0.8053
gini 1000 0.6819
spatial gini 1000 0.7316
 
CPU times: total: 3min 46s
Wall time: 3min 46s


In [119]:
gini_df = pd.DataFrame(cities_gini, columns = ['Gini E-300','Gini E-600','Gini E-1000'])
spatialgini_df = pd.DataFrame(cities_spatialgini, columns = ['S-gini E-300','S-gini E-600','S-gini E-1000'])
Gcomb_gini = pd.concat([gini_df, spatialgini_df], axis = 1)
Gcomb_gini.index = [i.split('_')[-1].split('.')[0] for i in ls_file]
Gcomb_gini

,Gini E-300,Gini E-600,Gini E-1000,S-gini E-300,S-gini E-600,S-gini E-1000
Canberra,0.847729,0.759195,0.687979,0.821566,0.739383,0.673909
Karlsruhe,0.695399,0.595181,0.505731,0.667862,0.607083,0.548317
Oslo,0.870242,0.769926,0.681926,0.886350,0.805276,0.731582


In [120]:
gini_comb = pd.concat([comb_gini, Gcomb_gini], axis = 1)
gini_comb.to_csv('D:/Dumps/GEE-WP Scores/gini_all_2.csv')
gini_comb

,Gini G-300,Gini G-600,Gini G-1000,S-gini G-300,S-gini G-600,S-gini G-1000,Gini E-300,Gini E-600,Gini E-1000,S-gini E-300,S-gini E-600,S-gini E-1000
Canberra,0.757310,0.674484,0.571259,0.715117,0.647454,0.559838,0.847729,0.759195,0.687979,0.821566,0.739383,0.673909
Karlsruhe,0.759177,0.621880,0.497628,0.687741,0.578448,0.482046,0.695399,0.595181,0.505731,0.667862,0.607083,0.548317
Oslo,0.875084,0.721834,0.583930,0.876949,0.729761,0.595352,0.870242,0.769926,0.681926,0.886350,0.805276,0.731582


In [121]:
Gcomb_gini.to_csv('D:/Dumps/GEE-WP Scores/E2SFCA/gini_scores_2.csv')